###Library

In [ ]:
import pandas as pd
import re
from datetime import datetime
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv("Housing.csv")

In [ ]:
df.head()

,WO #,Requested,Task Closed,Finish Date,Est. Completion,Facility,Building,Area,Department,Task Code-Description,...,Contractor,Status,Project,Project Name,Request,Due Date,WO Type Description,WO Task Posted Hours,Chargeable,Item SubLocation
0,HS-30492,10/12/2019 15:58,08/27/2021 14:34,08/27/2021 14:34,NaN,University Park Campus,McCarthy Honors College Dining,UVF-HS-1200A,HOSP. CSC,APP032 - Commercial Freezer Repair,...,NaN,Completed,NaN,NaN,"***Scheduled for"" Christmas break""****\n****sc...",NaN,Corrective Maintenance,14.49,False,NaN
1,HS-31528,02/20/2020 11:34,01/25/2021 14:52,01/25/2021 14:52,NaN,University Park Campus,Parkside Restaurant at Internationa,IRC-HS-1116,HOSP. CSC,ELE007 - Switches Repair/Replacement,...,Globis Electrical Corporation,Completed,NaN,NaN,Some of the light switches on control panel ar...,NaN,Corrective Maintenance,NaN,False,NaN
2,HS-31682,03/13/2020 14:06,02/17/2021 11:00,02/17/2021 11:00,NaN,Health Science Campus,Einstein Bros. Bagels at Zilkha Neu,ZNI-HS-100A,HOSP. CSC,APP032 - Commercial Freezer Repair,...,NaN,Completed,NaN,NaN,A work order from building manger was placed a...,NaN,Corrective Maintenance,1.65,False,NaN
3,HS-31893,07/29/2020 11:52,01/26/2021 12:46,01/21/2021 12:38,NaN,University Park Campus,UNIVERSITY CLUB AT KING STOOPS HALL,KSH-HS-101B,HOSP. CSC,CAR004 - Cabinet Repair,...,Custom Wood Display,Completed,NaN,NaN,Update: Estimate for water damage on \ncounter...,NaN,Corrective Maintenance,NaN,False,NaN
4,HS-31971,09/02/2020 12:24,01/06/2021 13:30,01/06/2021 13:30,NaN,University Park Campus,Everybody's Kitchen at EVK,EVK-HS-1123C,HOSP. CSC,APP002 - Refrigerator Repair,...,NaN,Completed,NaN,NaN,"Big walk needs to colder, according the health...",NaN,Corrective Maintenance,51.75,False,EVK RECEIVING


##Pre-processing and Feature Engineering

###Nan Values

In [ ]:
data = df
data.shape

(53913, 28)

In [ ]:
data.isna().sum()

WO #                         0
Requested                    0
Task Closed                  0
Finish Date              13724
Est. Completion          52726
Facility                     0
Building                     0
Area                         0
Department                   0
Task Code-Description        0
Type-Description             0
Tag #                        0
Description                  0
Item Type                    0
Shop                     12671
Trade                        0
Supervisor               53898
Priority                     0
Contractor               41391
Status                       0
Project                  53232
Project Name             53232
Request                      0
Due Date                 53783
WO Type Description          0
WO Task Posted Hours     21351
Chargeable                   0
Item SubLocation         52919
dtype: int64

In [ ]:
data = data.drop(["Finish Date","Shop","Est. Completion","Supervisor","Contractor","Project","Project Name","Due Date","WO Task Posted Hours","Item SubLocation"], axis = 1)  

In [ ]:
data.isna().sum()

WO #                     0
Requested                0
Task Closed              0
Facility                 0
Building                 0
Area                     0
Department               0
Task Code-Description    0
Type-Description         0
Tag #                    0
Description              0
Item Type                0
Trade                    0
Priority                 0
Status                   0
Request                  0
WO Type Description      0
Chargeable               0
dtype: int64

###Converting Date

In [ ]:
data.head()

,WO #,Requested,Task Closed,Facility,Building,Area,Department,Task Code-Description,Type-Description,Tag #,Description,Item Type,Trade,Priority,Status,Request,WO Type Description,Chargeable
0,HS-30492,10/12/2019 15:58,08/27/2021 14:34,University Park Campus,McCarthy Honors College Dining,UVF-HS-1200A,HOSP. CSC,APP032 - Commercial Freezer Repair,Appliances - Appliances,UVF-HS-1200A,Salad/Deli/Soup,Area,THSM GMT,Scheduled,Completed,"***Scheduled for"" Christmas break""****\n****sc...",Corrective Maintenance,False
1,HS-31528,02/20/2020 11:34,01/25/2021 14:52,University Park Campus,Parkside Restaurant at Internationa,IRC-HS-1116,HOSP. CSC,ELE007 - Switches Repair/Replacement,Electrical - Electrical,IRC-HS-1116,Parkside Rest. Dining Area,Area,OUTSIDE VENDOR,Routine,Completed,Some of the light switches on control panel ar...,Corrective Maintenance,False
2,HS-31682,03/13/2020 14:06,02/17/2021 11:00,Health Science Campus,Einstein Bros. Bagels at Zilkha Neu,ZNI-HS-100A,HOSP. CSC,APP032 - Commercial Freezer Repair,Appliances - Appliances,9402,Freezer Walk In,Equipment,THSM GMT,Routine,Completed,A work order from building manger was placed a...,Corrective Maintenance,False
3,HS-31893,07/29/2020 11:52,01/26/2021 12:46,University Park Campus,UNIVERSITY CLUB AT KING STOOPS HALL,KSH-HS-101B,HOSP. CSC,CAR004 - Cabinet Repair,Carpentry - Carpentry,KSH-HS-101B,101B-Main Dining-2,Area,OUTSIDE VENDOR,Scheduled,Completed,Update: Estimate for water damage on \ncounter...,Corrective Maintenance,False
4,HS-31971,09/02/2020 12:24,01/06/2021 13:30,University Park Campus,Everybody's Kitchen at EVK,EVK-HS-1123C,HOSP. CSC,APP002 - Refrigerator Repair,Appliances - Appliances,8010,REFRIGERATOR WALK IN 544 SQ FEET,Equipment,THSM GMT,Routine,Completed,"Big walk needs to colder, according the health...",Corrective Maintenance,False


In [ ]:
"""from datetime import datetime

datetime_str = '09/19/2021'

datetime_object = datetime.strptime(datetime_str, '%m/%d/%Y')

print(type(datetime_object))
print(datetime_object) """

"from datetime import datetime\n\ndatetime_str = '09/19/2021'\n\ndatetime_object = datetime.strptime(datetime_str, '%m/%d/%Y')\n\nprint(type(datetime_object))\nprint(datetime_object) "

In [ ]:
def convert_date(date):
  d = date.split(" ")[0]
  obj = str(datetime.strptime(d, '%m/%d/%Y')).split(" ")[0]
  return obj

In [ ]:
data["Requested"] = data["Requested"].apply(lambda x : convert_date(x))

In [ ]:
data["Task Closed"] = data["Task Closed"].apply(lambda x : convert_date(x))

In [ ]:
data["Requested"] = data["Requested"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))

In [ ]:
data["Task Closed"] = data["Task Closed"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))

In [ ]:
data.head()

,WO #,Requested,Task Closed,Facility,Building,Area,Department,Task Code-Description,Type-Description,Tag #,Description,Item Type,Trade,Priority,Status,Request,WO Type Description,Chargeable
0,HS-30492,2019-10-12,2021-08-27,University Park Campus,McCarthy Honors College Dining,UVF-HS-1200A,HOSP. CSC,APP032 - Commercial Freezer Repair,Appliances - Appliances,UVF-HS-1200A,Salad/Deli/Soup,Area,THSM GMT,Scheduled,Completed,"***Scheduled for"" Christmas break""****\n****sc...",Corrective Maintenance,False
1,HS-31528,2020-02-20,2021-01-25,University Park Campus,Parkside Restaurant at Internationa,IRC-HS-1116,HOSP. CSC,ELE007 - Switches Repair/Replacement,Electrical - Electrical,IRC-HS-1116,Parkside Rest. Dining Area,Area,OUTSIDE VENDOR,Routine,Completed,Some of the light switches on control panel ar...,Corrective Maintenance,False
2,HS-31682,2020-03-13,2021-02-17,Health Science Campus,Einstein Bros. Bagels at Zilkha Neu,ZNI-HS-100A,HOSP. CSC,APP032 - Commercial Freezer Repair,Appliances - Appliances,9402,Freezer Walk In,Equipment,THSM GMT,Routine,Completed,A work order from building manger was placed a...,Corrective Maintenance,False
3,HS-31893,2020-07-29,2021-01-26,University Park Campus,UNIVERSITY CLUB AT KING STOOPS HALL,KSH-HS-101B,HOSP. CSC,CAR004 - Cabinet Repair,Carpentry - Carpentry,KSH-HS-101B,101B-Main Dining-2,Area,OUTSIDE VENDOR,Scheduled,Completed,Update: Estimate for water damage on \ncounter...,Corrective Maintenance,False
4,HS-31971,2020-09-02,2021-01-06,University Park Campus,Everybody's Kitchen at EVK,EVK-HS-1123C,HOSP. CSC,APP002 - Refrigerator Repair,Appliances - Appliances,8010,REFRIGERATOR WALK IN 544 SQ FEET,Equipment,THSM GMT,Routine,Completed,"Big walk needs to colder, according the health...",Corrective Maintenance,False


###Converting WO #

In [ ]:
data["WO Code"]= data["WO #"].apply(lambda x : x.split("-")[0])
data["WO #"] = data["WO #"].apply(lambda x : x.split("-")[1])

In [ ]:
data["WO Code"]

0        HS
1        HS
2        HS
3        HS
4        HS
         ..
53908    TH
53909    TH
53910    TH
53911    TH
53912    TH
Name: WO Code, Length: 53913, dtype: object

In [ ]:
def conv_code(code):
  if code =="HS":
    code = 0
  else:
    code = 1
  return code

In [ ]:
data["WO Code"] = data["WO Code"].apply(lambda x : conv_code(x))

In [ ]:
data["WO Code"][0]

0

In [ ]:
data.head()

,WO #,Requested,Task Closed,Facility,Building,Area,Department,Task Code-Description,Type-Description,Tag #,Description,Item Type,Trade,Priority,Status,Request,WO Type Description,Chargeable,WO Code
0,30492,2019-10-12,2021-08-27,University Park Campus,McCarthy Honors College Dining,UVF-HS-1200A,HOSP. CSC,APP032 - Commercial Freezer Repair,Appliances - Appliances,UVF-HS-1200A,Salad/Deli/Soup,Area,THSM GMT,Scheduled,Completed,"***Scheduled for"" Christmas break""****\n****sc...",Corrective Maintenance,False,0
1,31528,2020-02-20,2021-01-25,University Park Campus,Parkside Restaurant at Internationa,IRC-HS-1116,HOSP. CSC,ELE007 - Switches Repair/Replacement,Electrical - Electrical,IRC-HS-1116,Parkside Rest. Dining Area,Area,OUTSIDE VENDOR,Routine,Completed,Some of the light switches on control panel ar...,Corrective Maintenance,False,0
2,31682,2020-03-13,2021-02-17,Health Science Campus,Einstein Bros. Bagels at Zilkha Neu,ZNI-HS-100A,HOSP. CSC,APP032 - Commercial Freezer Repair,Appliances - Appliances,9402,Freezer Walk In,Equipment,THSM GMT,Routine,Completed,A work order from building manger was placed a...,Corrective Maintenance,False,0
3,31893,2020-07-29,2021-01-26,University Park Campus,UNIVERSITY CLUB AT KING STOOPS HALL,KSH-HS-101B,HOSP. CSC,CAR004 - Cabinet Repair,Carpentry - Carpentry,KSH-HS-101B,101B-Main Dining-2,Area,OUTSIDE VENDOR,Scheduled,Completed,Update: Estimate for water damage on \ncounter...,Corrective Maintenance,False,0
4,31971,2020-09-02,2021-01-06,University Park Campus,Everybody's Kitchen at EVK,EVK-HS-1123C,HOSP. CSC,APP002 - Refrigerator Repair,Appliances - Appliances,8010,REFRIGERATOR WALK IN 544 SQ FEET,Equipment,THSM GMT,Routine,Completed,"Big walk needs to colder, according the health...",Corrective Maintenance,False,0


###Facility

In [ ]:
data["Facility"].value_counts()

University Park Campus    53756
Health Science Campus       157
Name: Facility, dtype: int64

In [ ]:
def conv_facility(f):
  if f == "University Park Campus":
    f = 0
  else : 
    f = 1
  return f

In [ ]:
data["Facility"] = data["Facility"].apply(lambda x : conv_facility(x))

In [ ]:
data["Facility"].head()

0    0
1    0
2    1
3    0
4    0
Name: Facility, dtype: int64

In [ ]:
data.head()

,WO #,Requested,Task Closed,Facility,Building,Area,Department,Task Code-Description,Type-Description,Tag #,Description,Item Type,Trade,Priority,Status,Request,WO Type Description,Chargeable,WO Code
0,30492,2019-10-12,2021-08-27,0,McCarthy Honors College Dining,UVF-HS-1200A,HOSP. CSC,APP032 - Commercial Freezer Repair,Appliances - Appliances,UVF-HS-1200A,Salad/Deli/Soup,Area,THSM GMT,Scheduled,Completed,"***Scheduled for"" Christmas break""****\n****sc...",Corrective Maintenance,False,0
1,31528,2020-02-20,2021-01-25,0,Parkside Restaurant at Internationa,IRC-HS-1116,HOSP. CSC,ELE007 - Switches Repair/Replacement,Electrical - Electrical,IRC-HS-1116,Parkside Rest. Dining Area,Area,OUTSIDE VENDOR,Routine,Completed,Some of the light switches on control panel ar...,Corrective Maintenance,False,0
2,31682,2020-03-13,2021-02-17,1,Einstein Bros. Bagels at Zilkha Neu,ZNI-HS-100A,HOSP. CSC,APP032 - Commercial Freezer Repair,Appliances - Appliances,9402,Freezer Walk In,Equipment,THSM GMT,Routine,Completed,A work order from building manger was placed a...,Corrective Maintenance,False,0
3,31893,2020-07-29,2021-01-26,0,UNIVERSITY CLUB AT KING STOOPS HALL,KSH-HS-101B,HOSP. CSC,CAR004 - Cabinet Repair,Carpentry - Carpentry,KSH-HS-101B,101B-Main Dining-2,Area,OUTSIDE VENDOR,Scheduled,Completed,Update: Estimate for water damage on \ncounter...,Corrective Maintenance,False,0
4,31971,2020-09-02,2021-01-06,0,Everybody's Kitchen at EVK,EVK-HS-1123C,HOSP. CSC,APP002 - Refrigerator Repair,Appliances - Appliances,8010,REFRIGERATOR WALK IN 544 SQ FEET,Equipment,THSM GMT,Routine,Completed,"Big walk needs to colder, according the health...",Corrective Maintenance,False,0


###Building and Area

In [ ]:
data.head()

,WO #,Requested,Task Closed,Facility,Building,Area,Department,Task Code-Description,Type-Description,Tag #,Description,Item Type,Trade,Priority,Status,Request,WO Type Description,Chargeable,WO Code
0,30492,2019-10-12,2021-08-27,0,McCarthy Honors College Dining,UVF-HS-1200A,HOSP. CSC,APP032 - Commercial Freezer Repair,Appliances - Appliances,UVF-HS-1200A,Salad/Deli/Soup,Area,THSM GMT,Scheduled,Completed,"***Scheduled for"" Christmas break""****\n****sc...",Corrective Maintenance,False,0
1,31528,2020-02-20,2021-01-25,0,Parkside Restaurant at Internationa,IRC-HS-1116,HOSP. CSC,ELE007 - Switches Repair/Replacement,Electrical - Electrical,IRC-HS-1116,Parkside Rest. Dining Area,Area,OUTSIDE VENDOR,Routine,Completed,Some of the light switches on control panel ar...,Corrective Maintenance,False,0
2,31682,2020-03-13,2021-02-17,1,Einstein Bros. Bagels at Zilkha Neu,ZNI-HS-100A,HOSP. CSC,APP032 - Commercial Freezer Repair,Appliances - Appliances,9402,Freezer Walk In,Equipment,THSM GMT,Routine,Completed,A work order from building manger was placed a...,Corrective Maintenance,False,0
3,31893,2020-07-29,2021-01-26,0,UNIVERSITY CLUB AT KING STOOPS HALL,KSH-HS-101B,HOSP. CSC,CAR004 - Cabinet Repair,Carpentry - Carpentry,KSH-HS-101B,101B-Main Dining-2,Area,OUTSIDE VENDOR,Scheduled,Completed,Update: Estimate for water damage on \ncounter...,Corrective Maintenance,False,0
4,31971,2020-09-02,2021-01-06,0,Everybody's Kitchen at EVK,EVK-HS-1123C,HOSP. CSC,APP002 - Refrigerator Repair,Appliances - Appliances,8010,REFRIGERATOR WALK IN 544 SQ FEET,Equipment,THSM GMT,Routine,Completed,"Big walk needs to colder, according the health...",Corrective Maintenance,False,0


In [ ]:
data["Area"].value_counts()

UVF-HS-1300       270
TCC-HS-KITCHEN    229
TRO-THSM-102      193
TRO-THSM-BLDG     185
UPX-TH-STORAGE    161
                 ... 
IRC-3113H           1
IRC-3112C           1
IRC-2013D           1
IRC-2012A           1
IRC-3101E           1
Name: Area, Length: 6665, dtype: int64

In [ ]:
data["Area"] = data["Area"].apply(lambda x : x.split("-")[0])

In [ ]:
data["Area"].nunique()

80

In [ ]:
ord_enc = OrdinalEncoder()
data["Area_code"] = ord_enc.fit_transform(data[["Area"]])

In [ ]:
data = data.drop("Area", axis =1)
data = data.drop("Building", axis =1)

In [ ]:
data.head()

,WO #,Requested,Task Closed,Facility,Department,Task Code-Description,Type-Description,Tag #,Description,Item Type,Trade,Priority,Status,Request,WO Type Description,Chargeable,WO Code,Area_code
0,30492,2019-10-12,2021-08-27,0,HOSP. CSC,APP032 - Commercial Freezer Repair,Appliances - Appliances,UVF-HS-1200A,Salad/Deli/Soup,Area,THSM GMT,Scheduled,Completed,"***Scheduled for"" Christmas break""****\n****sc...",Corrective Maintenance,False,0,74.0
1,31528,2020-02-20,2021-01-25,0,HOSP. CSC,ELE007 - Switches Repair/Replacement,Electrical - Electrical,IRC-HS-1116,Parkside Rest. Dining Area,Area,OUTSIDE VENDOR,Routine,Completed,Some of the light switches on control panel ar...,Corrective Maintenance,False,0,29.0
2,31682,2020-03-13,2021-02-17,1,HOSP. CSC,APP032 - Commercial Freezer Repair,Appliances - Appliances,9402,Freezer Walk In,Equipment,THSM GMT,Routine,Completed,A work order from building manger was placed a...,Corrective Maintenance,False,0,79.0
3,31893,2020-07-29,2021-01-26,0,HOSP. CSC,CAR004 - Cabinet Repair,Carpentry - Carpentry,KSH-HS-101B,101B-Main Dining-2,Area,OUTSIDE VENDOR,Scheduled,Completed,Update: Estimate for water damage on \ncounter...,Corrective Maintenance,False,0,33.0
4,31971,2020-09-02,2021-01-06,0,HOSP. CSC,APP002 - Refrigerator Repair,Appliances - Appliances,8010,REFRIGERATOR WALK IN 544 SQ FEET,Equipment,THSM GMT,Routine,Completed,"Big walk needs to colder, according the health...",Corrective Maintenance,False,0,18.0


###Department

In [ ]:
data["Department"].value_counts()

SIE               12093
TRH                7039
CAR                6951
CAP                5536
UVN-CRC            3411
HOSP. CSC          3194
UVS-CIC            2993
IRC                2382
UVF-MHC            2233
NORTH              2108
UVE-NBC            1680
SOUTH              1494
PRB                1106
UVV-MRC             941
HSM                 662
FLT                  69
USH                  13
ADMIN                 4
TRANSPORTATION        3
UVRET                 1
Name: Department, dtype: int64

In [ ]:
ord_enc = OrdinalEncoder()
data["Dep_code"] = ord_enc.fit_transform(data[["Department"]])

In [ ]:
data = data.drop("Department", axis =1)

In [ ]:
data.head()

,WO #,Requested,Task Closed,Facility,Task Code-Description,Type-Description,Tag #,Description,Item Type,Trade,Priority,Status,Request,WO Type Description,Chargeable,WO Code,Area_code,Dep_code
0,30492,2019-10-12,2021-08-27,0,APP032 - Commercial Freezer Repair,Appliances - Appliances,UVF-HS-1200A,Salad/Deli/Soup,Area,THSM GMT,Scheduled,Completed,"***Scheduled for"" Christmas break""****\n****sc...",Corrective Maintenance,False,0,74.0,4.0
1,31528,2020-02-20,2021-01-25,0,ELE007 - Switches Repair/Replacement,Electrical - Electrical,IRC-HS-1116,Parkside Rest. Dining Area,Area,OUTSIDE VENDOR,Routine,Completed,Some of the light switches on control panel ar...,Corrective Maintenance,False,0,29.0,4.0
2,31682,2020-03-13,2021-02-17,1,APP032 - Commercial Freezer Repair,Appliances - Appliances,9402,Freezer Walk In,Equipment,THSM GMT,Routine,Completed,A work order from building manger was placed a...,Corrective Maintenance,False,0,79.0,4.0
3,31893,2020-07-29,2021-01-26,0,CAR004 - Cabinet Repair,Carpentry - Carpentry,KSH-HS-101B,101B-Main Dining-2,Area,OUTSIDE VENDOR,Scheduled,Completed,Update: Estimate for water damage on \ncounter...,Corrective Maintenance,False,0,33.0,4.0
4,31971,2020-09-02,2021-01-06,0,APP002 - Refrigerator Repair,Appliances - Appliances,8010,REFRIGERATOR WALK IN 544 SQ FEET,Equipment,THSM GMT,Routine,Completed,"Big walk needs to colder, according the health...",Corrective Maintenance,False,0,18.0,4.0


###Task Code-Description

In [ ]:
data.head()

,WO #,Requested,Task Closed,Facility,Task Code-Description,Type-Description,Tag #,Description,Item Type,Trade,Priority,Status,Request,WO Type Description,Chargeable,WO Code,Area_code,Dep_code
0,30492,2019-10-12,2021-08-27,0,APP032 - Commercial Freezer Repair,Appliances - Appliances,UVF-HS-1200A,Salad/Deli/Soup,Area,THSM GMT,Scheduled,Completed,"***Scheduled for"" Christmas break""****\n****sc...",Corrective Maintenance,False,0,74.0,4.0
1,31528,2020-02-20,2021-01-25,0,ELE007 - Switches Repair/Replacement,Electrical - Electrical,IRC-HS-1116,Parkside Rest. Dining Area,Area,OUTSIDE VENDOR,Routine,Completed,Some of the light switches on control panel ar...,Corrective Maintenance,False,0,29.0,4.0
2,31682,2020-03-13,2021-02-17,1,APP032 - Commercial Freezer Repair,Appliances - Appliances,9402,Freezer Walk In,Equipment,THSM GMT,Routine,Completed,A work order from building manger was placed a...,Corrective Maintenance,False,0,79.0,4.0
3,31893,2020-07-29,2021-01-26,0,CAR004 - Cabinet Repair,Carpentry - Carpentry,KSH-HS-101B,101B-Main Dining-2,Area,OUTSIDE VENDOR,Scheduled,Completed,Update: Estimate for water damage on \ncounter...,Corrective Maintenance,False,0,33.0,4.0
4,31971,2020-09-02,2021-01-06,0,APP002 - Refrigerator Repair,Appliances - Appliances,8010,REFRIGERATOR WALK IN 544 SQ FEET,Equipment,THSM GMT,Routine,Completed,"Big walk needs to colder, according the health...",Corrective Maintenance,False,0,18.0,4.0


In [ ]:
data["Task Code-Description"].nunique()

271

In [ ]:
ord_enc = OrdinalEncoder()
data["Task_Code"]= ord_enc.fit_transform(data[["Task Code-Description"]])

In [ ]:
data = data.drop("Task Code-Description",axis =1)
data["Task_Code"].nunique()


271

In [ ]:
data.head()

,WO #,Requested,Task Closed,Facility,Type-Description,Tag #,Description,Item Type,Trade,Priority,Status,Request,WO Type Description,Chargeable,WO Code,Area_code,Dep_code,Task_Code
0,30492,2019-10-12,2021-08-27,0,Appliances - Appliances,UVF-HS-1200A,Salad/Deli/Soup,Area,THSM GMT,Scheduled,Completed,"***Scheduled for"" Christmas break""****\n****sc...",Corrective Maintenance,False,0,74.0,4.0,49.0
1,31528,2020-02-20,2021-01-25,0,Electrical - Electrical,IRC-HS-1116,Parkside Rest. Dining Area,Area,OUTSIDE VENDOR,Routine,Completed,Some of the light switches on control panel ar...,Corrective Maintenance,False,0,29.0,4.0,112.0
2,31682,2020-03-13,2021-02-17,1,Appliances - Appliances,9402,Freezer Walk In,Equipment,THSM GMT,Routine,Completed,A work order from building manger was placed a...,Corrective Maintenance,False,0,79.0,4.0,49.0
3,31893,2020-07-29,2021-01-26,0,Carpentry - Carpentry,KSH-HS-101B,101B-Main Dining-2,Area,OUTSIDE VENDOR,Scheduled,Completed,Update: Estimate for water damage on \ncounter...,Corrective Maintenance,False,0,33.0,4.0,73.0
4,31971,2020-09-02,2021-01-06,0,Appliances - Appliances,8010,REFRIGERATOR WALK IN 544 SQ FEET,Equipment,THSM GMT,Routine,Completed,"Big walk needs to colder, according the health...",Corrective Maintenance,False,0,18.0,4.0,23.0


In [ ]:
data = data.drop(["Type-Description", "Tag #", "Description"], axis = 1)

In [ ]:
data[["Item Type"]].value_counts()

Item Type
Area         51099
Equipment     1896
Asset          550
Vehicle        368
dtype: int64

In [ ]:
ord_enc = OrdinalEncoder()
data["Item_Type"]= ord_enc.fit_transform(data[["Item Type"]])

In [ ]:
data = data.drop("Item Type", axis = 1)

In [ ]:
data.head()

,WO #,Requested,Task Closed,Facility,Trade,Priority,Status,Request,WO Type Description,Chargeable,WO Code,Area_code,Dep_code,Task_Code,Item_Type
0,30492,2019-10-12,2021-08-27,0,THSM GMT,Scheduled,Completed,"***Scheduled for"" Christmas break""****\n****sc...",Corrective Maintenance,False,0,74.0,4.0,49.0,0.0
1,31528,2020-02-20,2021-01-25,0,OUTSIDE VENDOR,Routine,Completed,Some of the light switches on control panel ar...,Corrective Maintenance,False,0,29.0,4.0,112.0,0.0
2,31682,2020-03-13,2021-02-17,1,THSM GMT,Routine,Completed,A work order from building manger was placed a...,Corrective Maintenance,False,0,79.0,4.0,49.0,2.0
3,31893,2020-07-29,2021-01-26,0,OUTSIDE VENDOR,Scheduled,Completed,Update: Estimate for water damage on \ncounter...,Corrective Maintenance,False,0,33.0,4.0,73.0,0.0
4,31971,2020-09-02,2021-01-06,0,THSM GMT,Routine,Completed,"Big walk needs to colder, according the health...",Corrective Maintenance,False,0,18.0,4.0,23.0,2.0


###Trade

In [ ]:
ord_enc = OrdinalEncoder()
data["Trade_Code"]= ord_enc.fit_transform(data[["Trade"]])

In [ ]:
data = data.drop("Trade", axis =1 )
data["Trade_Code"].nunique()

39

###Priority

In [ ]:
data.head()

,WO #,Requested,Task Closed,Facility,Priority,Status,Request,WO Type Description,Chargeable,WO Code,Area_code,Dep_code,Task_Code,Item_Type,Trade_Code
0,30492,2019-10-12,2021-08-27,0,Scheduled,Completed,"***Scheduled for"" Christmas break""****\n****sc...",Corrective Maintenance,False,0,74.0,4.0,49.0,0.0,24.0
1,31528,2020-02-20,2021-01-25,0,Routine,Completed,Some of the light switches on control panel ar...,Corrective Maintenance,False,0,29.0,4.0,112.0,0.0,15.0
2,31682,2020-03-13,2021-02-17,1,Routine,Completed,A work order from building manger was placed a...,Corrective Maintenance,False,0,79.0,4.0,49.0,2.0,24.0
3,31893,2020-07-29,2021-01-26,0,Scheduled,Completed,Update: Estimate for water damage on \ncounter...,Corrective Maintenance,False,0,33.0,4.0,73.0,0.0,15.0
4,31971,2020-09-02,2021-01-06,0,Routine,Completed,"Big walk needs to colder, according the health...",Corrective Maintenance,False,0,18.0,4.0,23.0,2.0,24.0


In [ ]:
print(data["Priority"].nunique())
data["Priority"].value_counts()

4


Routine      47434
Urgent        5113
Scheduled     1238
Emergency      128
Name: Priority, dtype: int64

In [ ]:
ord_enc = OrdinalEncoder()
data["Priority_Code"]= ord_enc.fit_transform(data[["Priority"]])

In [ ]:
data = data.drop("Priority",axis =1)

In [ ]:
data.head()

,WO #,Requested,Task Closed,Facility,Status,Request,WO Type Description,Chargeable,WO Code,Area_code,Dep_code,Task_Code,Item_Type,Trade_Code,Priority_Code
0,30492,2019-10-12,2021-08-27,0,Completed,"***Scheduled for"" Christmas break""****\n****sc...",Corrective Maintenance,False,0,74.0,4.0,49.0,0.0,24.0,2.0
1,31528,2020-02-20,2021-01-25,0,Completed,Some of the light switches on control panel ar...,Corrective Maintenance,False,0,29.0,4.0,112.0,0.0,15.0,1.0
2,31682,2020-03-13,2021-02-17,1,Completed,A work order from building manger was placed a...,Corrective Maintenance,False,0,79.0,4.0,49.0,2.0,24.0,1.0
3,31893,2020-07-29,2021-01-26,0,Completed,Update: Estimate for water damage on \ncounter...,Corrective Maintenance,False,0,33.0,4.0,73.0,0.0,15.0,2.0
4,31971,2020-09-02,2021-01-06,0,Completed,"Big walk needs to colder, according the health...",Corrective Maintenance,False,0,18.0,4.0,23.0,2.0,24.0,1.0


In [ ]:
data["Status"].nunique()

6

In [ ]:
data["Status"].value_counts()

Completed        52702
Finished          1158
Created             37
Not Finished        14
Reassign             1
Not Completed        1
Name: Status, dtype: int64

In [ ]:
ord_enc = OrdinalEncoder()
data["Status_Code"]= ord_enc.fit_transform(data[["Status"]])
data = data.drop("Status",axis =1)

In [ ]:
data["WO Type Description"].nunique()
data["WO Type Description"].value_counts()

Corrective Maintenance    53265
PM (Preventive Maint.)      234
PM Repair                   206
Project                     153
Service Request              48
Scheduled Work                7
Name: WO Type Description, dtype: int64

In [ ]:
ord_enc = OrdinalEncoder()
data["WO_Type_Description"]= ord_enc.fit_transform(data[["WO Type Description"]])
data = data.drop("WO Type Description",axis =1)

In [ ]:
data.head()

,WO #,Requested,Task Closed,Facility,Request,Chargeable,WO Code,Area_code,Dep_code,Task_Code,Item_Type,Trade_Code,Priority_Code,Status_Code,WO_Type_Description
0,30492,2019-10-12,2021-08-27,0,"***Scheduled for"" Christmas break""****\n****sc...",False,0,74.0,4.0,49.0,0.0,24.0,2.0,0.0,0.0
1,31528,2020-02-20,2021-01-25,0,Some of the light switches on control panel ar...,False,0,29.0,4.0,112.0,0.0,15.0,1.0,0.0,0.0
2,31682,2020-03-13,2021-02-17,1,A work order from building manger was placed a...,False,0,79.0,4.0,49.0,2.0,24.0,1.0,0.0,0.0
3,31893,2020-07-29,2021-01-26,0,Update: Estimate for water damage on \ncounter...,False,0,33.0,4.0,73.0,0.0,15.0,2.0,0.0,0.0
4,31971,2020-09-02,2021-01-06,0,"Big walk needs to colder, according the health...",False,0,18.0,4.0,23.0,2.0,24.0,1.0,0.0,0.0


In [ ]:
l = list()
for i in range(len(data["Task Closed"])):
  l.append(int(str(data["Task Closed"][i] - data["Requested"][i]).split(" ")[0]))
  
data["Days"] = l

In [ ]:
data.head()

,WO #,Requested,Task Closed,Facility,Request,Chargeable,WO Code,Area_code,Dep_code,Task_Code,Item_Type,Trade_Code,Priority_Code,Status_Code,WO_Type_Description,Days
0,30492,2019-10-12,2021-08-27,0,"***Scheduled for"" Christmas break""****\n****sc...",False,0,74.0,4.0,49.0,0.0,24.0,2.0,0.0,0.0,685
1,31528,2020-02-20,2021-01-25,0,Some of the light switches on control panel ar...,False,0,29.0,4.0,112.0,0.0,15.0,1.0,0.0,0.0,340
2,31682,2020-03-13,2021-02-17,1,A work order from building manger was placed a...,False,0,79.0,4.0,49.0,2.0,24.0,1.0,0.0,0.0,341
3,31893,2020-07-29,2021-01-26,0,Update: Estimate for water damage on \ncounter...,False,0,33.0,4.0,73.0,0.0,15.0,2.0,0.0,0.0,181
4,31971,2020-09-02,2021-01-06,0,"Big walk needs to colder, according the health...",False,0,18.0,4.0,23.0,2.0,24.0,1.0,0.0,0.0,126


##Training

In [ ]:
data["Days"].nunique()

287

In [ ]:
y_train = data.iloc[:,-1].values
X_train = data.drop(["Requested","Task Closed", "Days","Request"],axis =1).iloc[:].values

In [ ]:
y_train.shape

(53913,)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size= 0.2)

In [ ]:
y_train.shape
X_train.shape
y_train.shape

(43130,)

In [ ]:
%%capture
gbr = GradientBoostingRegressor()
gbr.fit(X_train,y_train)

In [ ]:
def train(n_est, max, lr):
  gbr = GradientBoostingRegressor(n_estimators = n_est, max_depth = max, learning_rate= lr)
  gbr.fit(X_train,y_train)
  y_pred = gbr.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)
  print("n_est {} / max {} / lr {} / mse {}".format(n_est, max, lr, mse))

In [ ]:
for n_est in [10,20,30]:
  for max in [50, 100, 150]:
    for lr in [0.1, 0.01, 0.2, 0.02]:
      train(n_est, max, lr)

n_est 10 / max 50 / lr 0.1 / mse 357.6626611596732
n_est 10 / max 50 / lr 0.01 / mse 445.7034601136043
n_est 10 / max 50 / lr 0.2 / mse 407.54132111025865
n_est 10 / max 50 / lr 0.02 / mse 411.61994538195177
n_est 10 / max 100 / lr 0.1 / mse 357.50517015055425
n_est 10 / max 100 / lr 0.01 / mse 446.1394782465407
n_est 10 / max 100 / lr 0.2 / mse 408.5532154805133
n_est 10 / max 100 / lr 0.02 / mse 411.29217680829896
n_est 10 / max 150 / lr 0.1 / mse 356.8501928526947
n_est 10 / max 150 / lr 0.01 / mse 445.9839712808453
n_est 10 / max 150 / lr 0.2 / mse 411.634633865398
n_est 10 / max 150 / lr 0.02 / mse 411.93799199514575
n_est 20 / max 50 / lr 0.1 / mse 405.605505225212
n_est 20 / max 50 / lr 0.01 / mse 411.81038271389895
n_est 20 / max 50 / lr 0.2 / mse 445.81441539101877
n_est 20 / max 50 / lr 0.02 / mse 370.5438628662217
n_est 20 / max 100 / lr 0.1 / mse 408.0298351892482
n_est 20 / max 100 / lr 0.01 / mse 411.2860469199691
n_est 20 / max 100 / lr 0.2 / mse 449.04628344570193
n_est

In [ ]:
def train_rfr(n_est, max, lr):
  rfr = RandomForestRegressor(n_estimators = n_est, max_depth = max)
  rfr.fit(X_train,y_train)
  y_pred = rfr.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)
  print("n_est {} / max {} / lr {} / mse {} / y_pred {} / y_test {}".format(n_est, max, lr, mse, y_pred, y_test))

In [ ]:
for n_est in [10,20,30]:
  for max in [50, 100, 150]:
    for lr in [0.1, 0.01, 0.2, 0.02]:
      train_rfr(n_est, max, lr)

n_est 10 / max 50 / lr 0.1 / mse 283.2444208476306 / y_pred [ 3.9  1.6  0.9 ...  3.9 55.6 19.1] / y_test [ 1  0  0 ...  8 49  1]
n_est 10 / max 50 / lr 0.01 / mse 291.7997635166466 / y_pred [ 4.3  0.8  1.  ...  2.7 45.3 19.3] / y_test [ 1  0  0 ...  8 49  1]
n_est 10 / max 50 / lr 0.2 / mse 275.6256273764258 / y_pred [ 3.9  0.6  1.2 ...  4.8 40.1 17. ] / y_test [ 1  0  0 ...  8 49  1]
n_est 10 / max 50 / lr 0.02 / mse 296.49568023741074 / y_pred [ 4.6  1.3  1.1 ...  6.9 48.3 11.9] / y_test [ 1  0  0 ...  8 49  1]
n_est 10 / max 100 / lr 0.1 / mse 285.90988500417325 / y_pred [ 3.9  0.8  1.1 ...  2.7 56.7 24.2] / y_test [ 1  0  0 ...  8 49  1]
n_est 10 / max 100 / lr 0.01 / mse 289.5199387925438 / y_pred [ 4.3  1.5  1.1 ...  3.8 47.2 18.4] / y_test [ 1  0  0 ...  8 49  1]
n_est 10 / max 100 / lr 0.2 / mse 293.1460150236484 / y_pred [ 8.4  1.2  0.9 ...  5.9 53.5 13.1] / y_test [ 1  0  0 ...  8 49  1]
n_est 10 / max 100 / lr 0.02 / mse 291.33537234535845 / y_pred [ 3.6  1.   0.9 ...  2.8 4

In [ ]:
data["Request"][0]

'***Scheduled for" Christmas break"****\n****scheduled for 08/14/2021****\nCold top is not working properly. Only half of the surface gets cold. tag # 15940. UPDATE: Tag # 15940 not found in TMA.'